In [1]:
from qiskit import QuantumCircuit, QuantumRegister

import math

# the following function builds a gate (oracle) that transforms |x>|y> to |x>|y - cut(x)>
def energy_oracle(graph_laplacian: list[list[int]], digits: int):

    # setting up the circuit
    qregx = QuantumRegister(len(graph_laplacian),"q-x")
    qregy = QuantumRegister(digits,"q-y")
    qc = QuantumCircuit(qregx,qregy)

    # QFT
    for i in range(digits):
        qc.h(qregy[i])

        theta = math.pi/2
        for j in range(i + 1, digits):
            qc.cp(theta, qregy[j], qregy[i])
            theta /= 2

    for i in range(digits // 2):
        qc.swap(qregy[i],qregy[- (i + 1)])

    # Phasers
    for i in range(len(graph_laplacian)):
        theta = - graph_laplacian[i][i] * math.pi
        for j in range(digits):
            qc.cp(theta, qregx[i], qregy[j])
            theta /= 2

        for j in range(i + 1, len(graph_laplacian[i])):
            if graph_laplacian[i][j] != 0:
                theta = - graph_laplacian[i][j] * math.pi * 2
                for k in range(digits):
                    qc.mcp(theta, [qregx[i], qregx[j]], qregy[k])
                    theta /= 2

    # Inverse QFT
    for i in range(digits // 2):
        qc.swap(qregy[i],qregy[- (i + 1)])

    for i in range(digits - 1, - 1, - 1):
        qc.h(qregy[i])

        theta = - math.pi/2
        for j in range(i - 1, - 1, - 1):
            qc.cp(theta, qregy[j], qregy[i])
            theta /= 2

    print(qc)

    return qc.to_gate()

graph_laplacian = [[3, - 1, - 1, - 1], [- 1, 1, 0, 0], [- 1, 0, 1, 0], [- 1, 0, 0, 1]]
digits = 4

energy_oracle(graph_laplacian, digits)

                                                                            »
q-x_0: ─────────────────────────────────────────────────────────────────────»
                                                                            »
q-x_1: ─────────────────────────────────────────────────────────────────────»
                                                                            »
q-x_2: ─────────────────────────────────────────────────────────────────────»
                                                                            »
q-x_3: ─────────────────────────────────────────────────────────────────────»
       ┌───┐                                                                »
q-y_0: ┤ H ├─■────────■─────────────■───────────────────────────────────────»
       └───┘ │P(π/2)  │       ┌───┐ │                                       »
q-y_1: ──────■────────┼───────┤ H ├─┼────────■────────■─────────────────────»
                      │P(π/4) └───┘ │        │P(π/2)  │       ┌─

Instruction(name='circuit-114', num_qubits=8, num_clbits=0, params=[])